In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [2]:
import os
import numpy as np
from tqdm import tqdm
import cv2
training_data=[]
label=[0,1,2,3,4,5,6,7,8,9]
root = 'MNIST/trainingSet/trainingSet/'
min=len(os.listdir(root+str(0)))
for dir in label:
    path = os.path.join(root,str(dir))
    if len(os.listdir(path)) < min:
        min = len(os.listdir(path))
    
for dir in label:
    count = 0
    path = os.path.join(root,str(dir))
    for f in tqdm(os.listdir(path)):
        img = cv2.cvtColor(cv2.imread(os.path.join(path,f)),cv2.COLOR_BGR2GRAY)
        img = torch.tensor(img,dtype=torch.float32).view(-1,1,28,28).to(device)
        label = torch.tensor([np.eye(10)[dir]],dtype=torch.float32).to(device)
        training_data.append([img,label])
        count+=1
        if count == min:
            break

np.random.shuffle(training_data)
np.save("training_data.npy",training_data)

 91%|█████████ | 3794/4188 [00:02<00:00, 1365.13it/s]
C:\Users\ASUS\anaconda3\envs\gym_atari\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [35]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,128,7)
        self.norm1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128,64,5)
        self.norm2 = nn.BatchNorm2d(64)
#         self.conv3 = nn.Conv2d(64,32,3)
#         self.norm3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(64*3*3,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,64)
        self.fc4 = nn.Linear(64,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.norm1(F.max_pool2d(x,(2,2),stride=2))
        x = self.norm2(F.max_pool2d(self.conv2(x),(2,2),stride=2))
#         x = F.dropout2d(x,p=0.2,training=True )
#         x = self.norm3(F.max_pool2d(self.conv3(x),(2,2),stride=2))
        x = F.dropout2d(x,p=0.2,training=True )
        x = x.view(-1,64*3*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x),dim=1)
        
        
        return x

net = Net().to(device)

In [36]:
X=[]
Y=[]
for data in training_data:
    X.append(data[0]/255)
    Y.append(data[1])

In [37]:
X = torch.cat(X,dim=0)
Y = torch.cat(Y,dim=0) 

In [38]:
Y[0]

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0')

In [39]:
batch_size=32
epochs=4
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)  
for _ in range(epochs):
    for i in tqdm(range(0,len(X),batch_size)):
        train_batch_x = X[i:i+batch_size].to(device)
        train_batch_y = Y[i:i+batch_size].to(device)
        output = net(train_batch_x)
        net.zero_grad()
        loss = criterion(output, train_batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

  1%|          | 9/1186 [00:00<00:13, 88.23it/s]

tensor(1.4832e-05, device='cuda:0', grad_fn=<MseLossBackward>)


  1%|          | 10/1186 [00:00<00:12, 93.45it/s]

tensor(2.6575e-07, device='cuda:0', grad_fn=<MseLossBackward>)


  1%|          | 9/1186 [00:00<00:13, 86.53it/s]

tensor(2.8772e-05, device='cuda:0', grad_fn=<MseLossBackward>)


100%|██████████| 1186/1186 [00:14<00:00, 82.34it/s]

tensor(9.0036e-05, device='cuda:0', grad_fn=<MseLossBackward>)


In [40]:
import os
test_root='MNIST/testSet/testSet/'
img='img_18.jpg'
img = cv2.cvtColor(cv2.imread(os.path.join(test_root,img)),cv2.COLOR_BGR2GRAY)
img = torch.tensor(img,dtype=torch.float32).view(-1,1,28,28).to(device)/255
pred=net(img)

In [41]:
torch.argmax(pred)

tensor(1, device='cuda:0', grad_fn=<NotImplemented>)

In [11]:
img

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0039, 0.0118, 0.0000, 0.0000, 0.0157, 0.0078, 0.0000,
           0.0431, 0.0000, 0.0000, 0.0549, 0.0039, 0.0000, 0.0745, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0471, 0.0000, 0.0000, 0.0275, 0.0000, 0.0039, 0.0392,
           0.0000, 0.0078, 0.0078, 0.0627, 0.0000, 0.0118, 0.0118, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0275, 0.0314, 0.0000, 0.0314, 0.0000, 0.0000, 0.0314, 0.0000,
           0.0000, 0.0745, 0.0000, 0.0000, 0.0039, 0.0824, 0.0000, 0.0157,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0431, 

In [ ]:
training_data[0][0].type()